# Lab Assignment 8: Data Management Using `pandas`, Part 1
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will be working with the [2017 Workplace Health in America survey](https://www.cdc.gov/workplacehealthpromotion/survey/data.html) which was conducted by the Centers for Disease Control and Prevention. According to the survey's [guidence document](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Guidance-Document-for-Use-of-Public-Data-files-508.pdf):

> The Workplace Health in America (WHA) Survey gathered information from a cross-sectional, nationally representative sample of US worksites. The sample was drawn from the Dun & Bradstreet (D&B) database of all private and public employers in the United States with at least 10 employees. Like previous national surveys, the worksite served as the sampling unit rather than the companies or firms to which the worksites belonged. Worksites were selected using a stratified simple random sample (SRS) design, where the primary strata were ten multi-state regions defined by the Centers for Disease Control and Prevention (CDC), plus an additional stratum containing all hospital worksites. 

The data contain over 300 features that report the industry and type of company where the respondents are employed, what kind of health insurance and other health programs are offered, and other characteristics of the workplaces including whether employees are allowed to work from home and the gender and age makeup of the workforce. The data are full of interesting information, but in order to make use of the data a great deal of data manipulation is required first.

## Problem 0
Import the following libraries:

In [1]:
import numpy as np
import pandas as pd
import sidetable
import sqlite3
import warnings
warnings.filterwarnings('ignore')

## Problem 1
The raw data are stored in an ASCII file on the 2017 Workplace Health in America survey [homepage](https://www.cdc.gov/workplacehealthpromotion/survey/data.html). Load the raw data directly into Python without downloading the data onto your harddrive and display a dataframe with only the 14th, 28th, and 102nd rows of the data. [1 point]

In [2]:
df = pd.read_csv('https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/whpps_120717.csv', delimiter = '~')
df.iloc[[13, 27, 101]]

,OC1,OC3,HI1,HI2,HI3,HI4,HRA1,HRA1A,HRA1B,HRA1E,...,WL3_05,E1_09,Suppquex,Id,Region,CDC_Region,Industry,Size,Varstrata,"Finalwt_worksite,,,,"
13,3,1.0,2.0,3.0,2.0,1.0,1.0,3.0,3.0,1.0,...,NaN,NaN,1.0,1437.0,4.0,6.0,7.0,3.0,0.0,"47.793940929,,,,"
27,1,3.0,1.0,3.0,1.0,1.0,1.0,2.0,4.0,2.0,...,NaN,NaN,1.0,2501.0,2.0,4.0,7.0,8.0,0.0,"47.793940929,,,,"
101,2,1.0,1.0,3.0,2.0,1.0,1.0,2.0,4.0,2.0,...,NaN,NaN,2.0,12636.0,4.0,6.0,7.0,4.0,0.0,"47.793940929,,,,"


## Problem 2 
The data contain 301 columns. Create a new variable in Python's memory to store a working version of the data. In the working version, delete all of the columns except for the following:

* `Industry`: 7 Industry Categories with NAICS codes

* `Size`: 8 Employee Size Categories

* `OC3` Is your organization for profit, non-profit, government?

* `HI1` In general, do you offer full, partial or no payment of premiums for personal health insurance for full-time employees?

* `HI2` Over the past 12 months, were full-time employees asked to pay a larger proportion, smaller proportion or the same proportion of personal health insurance premiums?

* `HI3`: Does your organization offer personal health insurance for your part-time employees?

* `CP1`: Are there health education programs, which focus on skill development and lifestyle behavior change along with information dissemination and awareness building?

* `WL6`: Allow employees to work from home?

* Every column that begins `WD`, expressing the percentage of employees that have certain characteristics at the firm

[1 point]

In [3]:
data_frame = df.copy(deep = True)
data_frame = data_frame.drop([column_name for column_name in data_frame.columns if (column_name not in ['Industry', 'Size', 'OC3', 'HI1', 'HI2', 'HI3', 'CP1', 'WL6']) and (not column_name.startswith('WD'))], axis = 1)
data_frame = df.copy(deep = True)
data_frame = data_frame[['Industry', 'Size', 'OC3', 'HI1', 'HI2', 'HI3', 'CP1', 'WL6'] + [column_name for column_name in data_frame.columns if column_name.startswith('WD')]]
data_frame

,Industry,Size,OC3,HI1,HI2,HI3,CP1,WL6,WD1_1,WD1_2,WD2,WD3,WD4,WD5,WD6,WD7
0,7.0,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,7.0,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,7.0,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,7.0,4.0,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,7.0,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,6.0,5.0,4.0,1.0,3.0,1.0,1.0,99.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
2839,6.0,5.0,4.0,2.0,3.0,1.0,1.0,2.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0
2840,6.0,8.0,4.0,2.0,3.0,1.0,1.0,1.0,27.0,997.0,61.0,997.0,997.0,997.0,997.0,997.0
2841,6.0,8.0,4.0,2.0,3.0,1.0,2.0,99.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0


## Problem 3
The [codebook](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Datafile-Codebook-508.pdf) for the WHA data contain short descriptions of the meaning of each of the columns in the data. Use these descriptions to decide on better and more intuitive names for the columns in the working version of the data, and rename the columns accordingly. [1 point]

In [4]:
data_frame.columns = ['industry', 'employee_size', 'organization_type', 'premiums', 'premium_change', 'part_time_insurance_coverage', 'health_promotion', 'work_from_home', 'percent_under30', 'percent_over60', 'percent_female', 'percent_hourly', 'percent_atypical_shift', 'percent_remote', 'percent_union', 'percent_turnover']
data_frame

,industry,employee_size,organization_type,premiums,premium_change,part_time_insurance_coverage,health_promotion,work_from_home,percent_under30,percent_over60,percent_female,percent_hourly,percent_atypical_shift,percent_remote,percent_union,percent_turnover
0,7.0,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,7.0,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,7.0,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,7.0,4.0,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,7.0,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,6.0,5.0,4.0,1.0,3.0,1.0,1.0,99.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
2839,6.0,5.0,4.0,2.0,3.0,1.0,1.0,2.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0
2840,6.0,8.0,4.0,2.0,3.0,1.0,1.0,1.0,27.0,997.0,61.0,997.0,997.0,997.0,997.0,997.0
2841,6.0,8.0,4.0,2.0,3.0,1.0,2.0,99.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0


## Problem 4
Using the codebook and this [dictionary of NAICS industrial codes](https://www.naics.com/search-naics-codes-by-industry/), place descriptive labels on the categories of the industry column in the working data. [1 point]

In [5]:
industry_map = {
    1: 'resources_and_infrastructure',
    2: 'trade',
    3: 'pleasure',
    4: 'technology_economy_and_management',
    5: 'health_and_social',
    6: 'public_administration',
    7: 'hospitals'
}
data_frame.industry = data_frame.industry.map(industry_map)
data_frame

,industry,employee_size,organization_type,premiums,premium_change,part_time_insurance_coverage,health_promotion,work_from_home,percent_under30,percent_over60,percent_female,percent_hourly,percent_atypical_shift,percent_remote,percent_union,percent_turnover
0,hospitals,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,hospitals,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,hospitals,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,hospitals,4.0,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,hospitals,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,public_administration,5.0,4.0,1.0,3.0,1.0,1.0,99.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
2839,public_administration,5.0,4.0,2.0,3.0,1.0,1.0,2.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0
2840,public_administration,8.0,4.0,2.0,3.0,1.0,1.0,1.0,27.0,997.0,61.0,997.0,997.0,997.0,997.0,997.0
2841,public_administration,8.0,4.0,2.0,3.0,1.0,2.0,99.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0


## Problem 5
Using the codebook, recode the "size" column to have three categories: "Small" for workplaces with fewer than 100 employees, "Medium" for workplaces with at least 100 but fewer than 500 employees, and "Large" for companies with at least 500 employees. [Note: Python dataframes have an attribute `.size` that reports the space the dataframe takes up in memory. Don't confuse this attribute with the column named "Size" in the raw data.] [1 point]

In [6]:
employee_size_map = {
    1: 'Small',
    2: 'Small',
    3: 'Small',
    4: 'Medium',
    5: 'Medium',
    6: 'Large',
    7: 'Large',
    8: 'Large'
}
data_frame.employee_size = data_frame.employee_size.map(employee_size_map)
data_frame

,industry,employee_size,organization_type,premiums,premium_change,part_time_insurance_coverage,health_promotion,work_from_home,percent_under30,percent_over60,percent_female,percent_hourly,percent_atypical_shift,percent_remote,percent_union,percent_turnover
0,hospitals,Large,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,hospitals,Large,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,hospitals,Large,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,hospitals,Medium,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,hospitals,Medium,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,public_administration,Medium,4.0,1.0,3.0,1.0,1.0,99.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
2839,public_administration,Medium,4.0,2.0,3.0,1.0,1.0,2.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0
2840,public_administration,Large,4.0,2.0,3.0,1.0,1.0,1.0,27.0,997.0,61.0,997.0,997.0,997.0,997.0,997.0
2841,public_administration,Large,4.0,2.0,3.0,1.0,2.0,99.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0


## Problem 6
Use the codebook to write accurate and descriptive labels for each category for each categorical column in the working data. Then apply all of these labels to the data at once. Code "Legitimate Skip", "Don't know", "Refused", and "Blank" as missing values. [2 points]

In [7]:
data_frame = df.copy(deep = True)
data_frame = data_frame[['Industry', 'Size', 'OC3', 'HI1', 'HI2', 'HI3', 'CP1', 'WL6'] + [column_name for column_name in data_frame.columns if column_name.startswith('WD')]]
data_frame.columns = ['industry', 'employee_size', 'organization_type', 'premiums', 'premium_change', 'part_time_insurance_coverage', 'health_promotion', 'work_from_home', 'percent_under30', 'percent_over60', 'percent_female', 'percent_hourly', 'percent_atypical_shift', 'percent_remote', 'percent_union', 'percent_turnover']
industry_map = {
    1: 'resources_and_infrastructure',
    2: 'trade',
    3: 'pleasure',
    4: 'technology_economy_and_management',
    5: 'health_and_social',
    6: 'public_administration',
    7: 'hospitals'
}
employee_size_map = {
    1: 'Small',
    2: 'Small',
    3: 'Small',
    4: 'Medium',
    5: 'Medium',
    6: 'Large',
    7: 'Large',
    8: 'Large'
}
organization_type_map = {
    1: 'for_profit_public',
    2: 'for_profit_private',
    3: 'nonprofit',
    4: 'state_or_local_government',
    5: 'federal_government',
    6: 'other',
    97: np.NaN,# 'dont_know',
    98: np.NaN,# 'refusal',
    99: np.NaN# 'blank'
}
premiums_map = {
    1: 'full',
    2: 'partial',
    3: 'no',
    97: np.NaN,# 'dont_know',
    98: np.NaN,# 'refusal',
    99: np.NaN# 'blank'
}
premium_change_map = {
    1: 'larger',
    2: 'smaller',
    3: 'about_the_same',
    96: np.NaN,# 'legitimate_skip',
    97: np.NaN,# 'dont_know',
    98: np.NaN,# 'refusal',
    99: np.NaN# 'blank'
}
part_time_insurance_coverage_map = {
    1: 'yes',
    2: 'no',
    97: np.NaN,# 'dont_know',
    98: np.NaN,# 'refusal',
    99: np.NaN# 'blank'
}
health_promotion_map = {
    1: 'yes',
    2: 'no',
    97: np.NaN,# 'dont_know',
    98: np.NaN# 'refusal'
}
work_from_home_map = {
    1: 'yes',
    2: 'no',
    97: np.NaN,# 'dont_know',
    98: np.NaN,# 'refusal',
    99: np.NaN# 'blank'
}
map_of_maps = {
    'industry': industry_map,
    'employee_size': employee_size_map,
    'organization_type': organization_type_map,
    'premiums': premiums_map,
    'premium_change': premium_change_map,
    'part_time_insurance_coverage': part_time_insurance_coverage_map,
    'health_promotion': health_promotion_map,
    'work_from_home': work_from_home_map
}
data_frame = data_frame.replace(to_replace = map_of_maps)
data_frame

,industry,employee_size,organization_type,premiums,premium_change,part_time_insurance_coverage,health_promotion,work_from_home,percent_under30,percent_over60,percent_female,percent_hourly,percent_atypical_shift,percent_remote,percent_union,percent_turnover
0,hospitals,Large,nonprofit,partial,larger,no,yes,yes,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,hospitals,Large,nonprofit,partial,about_the_same,yes,yes,yes,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,hospitals,Large,nonprofit,full,about_the_same,yes,yes,yes,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,hospitals,Medium,for_profit_private,full,smaller,yes,no,no,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,hospitals,Medium,nonprofit,full,about_the_same,yes,yes,yes,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,public_administration,Medium,state_or_local_government,full,about_the_same,yes,yes,NaN,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
2839,public_administration,Medium,state_or_local_government,partial,about_the_same,yes,yes,no,997.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0
2840,public_administration,Large,state_or_local_government,partial,about_the_same,yes,yes,yes,27.0,997.0,61.0,997.0,997.0,997.0,997.0,997.0
2841,public_administration,Large,state_or_local_government,partial,about_the_same,yes,no,NaN,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0


## Problem 7
The features that measure the percent of the workforce with a particular characteristic use the codes 997, 998, and 999 to represent "Don't know", "Refusal", and "Blank/Invalid" respectively. Replace these values with missing values for all of the percentage features at the same time. [1 point]

In [8]:
missing_values_map = {
    997: np.NaN,
    998: np.NaN,
    999: np.NaN
}
data_frame = data_frame.replace(missing_values_map)
data_frame

,industry,employee_size,organization_type,premiums,premium_change,part_time_insurance_coverage,health_promotion,work_from_home,percent_under30,percent_over60,percent_female,percent_hourly,percent_atypical_shift,percent_remote,percent_union,percent_turnover
0,hospitals,Large,nonprofit,partial,larger,no,yes,yes,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,hospitals,Large,nonprofit,partial,about_the_same,yes,yes,yes,NaN,NaN,90.0,90.0,NaN,NaN,0.0,NaN
2,hospitals,Large,nonprofit,full,about_the_same,yes,yes,yes,35.0,4.0,NaN,NaN,40.0,15.0,NaN,NaN
3,hospitals,Medium,for_profit_private,full,smaller,yes,no,no,50.0,15.0,50.0,85.0,75.0,0.0,0.0,NaN
4,hospitals,Medium,nonprofit,full,about_the_same,yes,yes,yes,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,public_administration,Medium,state_or_local_government,full,about_the_same,yes,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2839,public_administration,Medium,state_or_local_government,partial,about_the_same,yes,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2840,public_administration,Large,state_or_local_government,partial,about_the_same,yes,yes,yes,27.0,NaN,61.0,NaN,NaN,NaN,NaN,NaN
2841,public_administration,Large,state_or_local_government,partial,about_the_same,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Problem 8
Sort the working data by industry in ascending alphabetical order. Within industry categories, sort the rows by size in ascending alphabetical order. Within groups with the same industry and size, sort by percent of the workforce that is under 30 in descending numeric order. [1 point]

In [9]:
data_frame = data_frame.sort_values(by = ['industry', 'employee_size', 'percent_under30'], ascending = [True, True, False])
data_frame

,industry,employee_size,organization_type,premiums,premium_change,part_time_insurance_coverage,health_promotion,work_from_home,percent_under30,percent_over60,percent_female,percent_hourly,percent_atypical_shift,percent_remote,percent_union,percent_turnover
1987,health_and_social,Large,nonprofit,partial,about_the_same,no,yes,yes,46.0,10.0,86.0,75.0,50.0,50.0,0.0,35.0
1988,health_and_social,Large,nonprofit,full,about_the_same,yes,yes,yes,45.0,25.0,65.0,60.0,60.0,5.0,NaN,7.0
1120,health_and_social,Large,state_or_local_government,full,about_the_same,no,yes,no,40.0,20.0,75.0,30.0,5.0,0.0,70.0,NaN
1641,health_and_social,Large,nonprofit,partial,about_the_same,yes,no,no,40.0,40.0,60.0,40.0,NaN,NaN,50.0,NaN
2807,health_and_social,Large,nonprofit,full,smaller,yes,no,no,40.0,20.0,65.0,60.0,100.0,5.0,0.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604,trade,Small,nonprofit,full,about_the_same,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2626,trade,Small,for_profit_private,partial,larger,yes,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2629,trade,Small,for_profit_public,full,larger,no,yes,yes,NaN,2.0,15.0,NaN,NaN,90.0,0.0,15.0
2631,trade,Small,for_profit_private,partial,larger,yes,no,no,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN


## Problem 9
There is one row in the working data that has a `NaN` value for industry. Delete this row. Use a logical expression, and not the row number. [1 point]

In [10]:
data_frame = data_frame.dropna(subset = ['industry'])
data_frame

,industry,employee_size,organization_type,premiums,premium_change,part_time_insurance_coverage,health_promotion,work_from_home,percent_under30,percent_over60,percent_female,percent_hourly,percent_atypical_shift,percent_remote,percent_union,percent_turnover
1987,health_and_social,Large,nonprofit,partial,about_the_same,no,yes,yes,46.0,10.0,86.0,75.0,50.0,50.0,0.0,35.0
1988,health_and_social,Large,nonprofit,full,about_the_same,yes,yes,yes,45.0,25.0,65.0,60.0,60.0,5.0,NaN,7.0
1120,health_and_social,Large,state_or_local_government,full,about_the_same,no,yes,no,40.0,20.0,75.0,30.0,5.0,0.0,70.0,NaN
1641,health_and_social,Large,nonprofit,partial,about_the_same,yes,no,no,40.0,40.0,60.0,40.0,NaN,NaN,50.0,NaN
2807,health_and_social,Large,nonprofit,full,smaller,yes,no,no,40.0,20.0,65.0,60.0,100.0,5.0,0.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,trade,Small,for_profit_private,full,about_the_same,yes,no,no,NaN,2.0,2.0,90.0,0.0,0.0,0.0,0.0
2604,trade,Small,nonprofit,full,about_the_same,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2626,trade,Small,for_profit_private,partial,larger,yes,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2629,trade,Small,for_profit_public,full,larger,no,yes,yes,NaN,2.0,15.0,NaN,NaN,90.0,0.0,15.0


## Problem 10
Create a new feature named `gender_balance` that has three categories: "Mostly men" for workplaces with between 0% and 35% female employees, "Balanced" for workplaces with more than 35% and at most 65% female employees, and "Mostly women" for workplaces with more than 65% female employees. [1 point]

In [11]:
data_frame['gender_balance'] = pd.cut(data_frame.percent_female, bins = [-0.1, 35, 65, 100], labels = ['Mostly men', 'Balanced', 'Mostly women'])
data_frame

,industry,employee_size,organization_type,premiums,premium_change,part_time_insurance_coverage,health_promotion,work_from_home,percent_under30,percent_over60,percent_female,percent_hourly,percent_atypical_shift,percent_remote,percent_union,percent_turnover,gender_balance
1987,health_and_social,Large,nonprofit,partial,about_the_same,no,yes,yes,46.0,10.0,86.0,75.0,50.0,50.0,0.0,35.0,Mostly women
1988,health_and_social,Large,nonprofit,full,about_the_same,yes,yes,yes,45.0,25.0,65.0,60.0,60.0,5.0,NaN,7.0,Balanced
1120,health_and_social,Large,state_or_local_government,full,about_the_same,no,yes,no,40.0,20.0,75.0,30.0,5.0,0.0,70.0,NaN,Mostly women
1641,health_and_social,Large,nonprofit,partial,about_the_same,yes,no,no,40.0,40.0,60.0,40.0,NaN,NaN,50.0,NaN,Balanced
2807,health_and_social,Large,nonprofit,full,smaller,yes,no,no,40.0,20.0,65.0,60.0,100.0,5.0,0.0,25.0,Balanced
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,trade,Small,for_profit_private,full,about_the_same,yes,no,no,NaN,2.0,2.0,90.0,0.0,0.0,0.0,0.0,Mostly men
2604,trade,Small,nonprofit,full,about_the_same,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2626,trade,Small,for_profit_private,partial,larger,yes,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2629,trade,Small,for_profit_public,full,larger,no,yes,yes,NaN,2.0,15.0,NaN,NaN,90.0,0.0,15.0,Mostly men


## Problem 11
Change the data type of all categorical features in the working data from "object" to "category". [1 point]

In [12]:
type_map = {
    'industry': 'category',
    'employee_size': 'category',
    'organization_type': 'category',
    'premiums': 'category',
    'premium_change': 'category',
    'part_time_insurance_coverage': 'category',
    'health_promotion': 'category',
    'work_from_home': 'category'
}
data_frame = data_frame.astype(type_map)
data_frame

,industry,employee_size,organization_type,premiums,premium_change,part_time_insurance_coverage,health_promotion,work_from_home,percent_under30,percent_over60,percent_female,percent_hourly,percent_atypical_shift,percent_remote,percent_union,percent_turnover,gender_balance
1987,health_and_social,Large,nonprofit,partial,about_the_same,no,yes,yes,46.0,10.0,86.0,75.0,50.0,50.0,0.0,35.0,Mostly women
1988,health_and_social,Large,nonprofit,full,about_the_same,yes,yes,yes,45.0,25.0,65.0,60.0,60.0,5.0,NaN,7.0,Balanced
1120,health_and_social,Large,state_or_local_government,full,about_the_same,no,yes,no,40.0,20.0,75.0,30.0,5.0,0.0,70.0,NaN,Mostly women
1641,health_and_social,Large,nonprofit,partial,about_the_same,yes,no,no,40.0,40.0,60.0,40.0,NaN,NaN,50.0,NaN,Balanced
2807,health_and_social,Large,nonprofit,full,smaller,yes,no,no,40.0,20.0,65.0,60.0,100.0,5.0,0.0,25.0,Balanced
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,trade,Small,for_profit_private,full,about_the_same,yes,no,no,NaN,2.0,2.0,90.0,0.0,0.0,0.0,0.0,Mostly men
2604,trade,Small,nonprofit,full,about_the_same,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2626,trade,Small,for_profit_private,partial,larger,yes,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2629,trade,Small,for_profit_public,full,larger,no,yes,yes,NaN,2.0,15.0,NaN,NaN,90.0,0.0,15.0,Mostly men


## Problem 12
Filter the data to only those rows that represent small workplaces that allow employees to work from home. Then report how many of these workplaces offer full insurance, partial insurance, and no insurance. Use a function that reports the percent, cumulative count, and cumulative percent in addition to the counts. [1 point]

In [13]:
data_frame_of_small_workplaces_that_allow_employees_to_work_from_home = data_frame[(data_frame.employee_size == 'Small') & (data_frame.work_from_home == 'yes')]
data_frame_of_small_workplaces_that_allow_employees_to_work_from_home

,industry,employee_size,organization_type,premiums,premium_change,part_time_insurance_coverage,health_promotion,work_from_home,percent_under30,percent_over60,percent_female,percent_hourly,percent_atypical_shift,percent_remote,percent_union,percent_turnover,gender_balance
1886,health_and_social,Small,nonprofit,full,about_the_same,no,yes,yes,95.0,0.0,90.0,95.0,100.0,75.0,0.0,10.0,Mostly women
2248,health_and_social,Small,nonprofit,partial,smaller,yes,no,yes,90.0,0.0,80.0,90.0,0.0,10.0,0.0,10.0,Mostly women
1952,health_and_social,Small,nonprofit,full,larger,no,no,yes,79.0,2.0,98.0,96.0,1.0,1.0,0.0,NaN,Mostly women
1385,health_and_social,Small,nonprofit,partial,larger,yes,no,yes,60.0,5.0,90.0,60.0,NaN,NaN,0.0,25.0,Mostly women
2259,health_and_social,Small,nonprofit,full,about_the_same,no,no,yes,60.0,12.0,90.0,72.0,4.0,0.0,0.0,NaN,Mostly women
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768,trade,Small,other,partial,NaN,yes,NaN,yes,NaN,NaN,15.0,80.0,NaN,NaN,NaN,NaN,Mostly men
2109,trade,Small,NaN,partial,smaller,no,no,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2112,trade,Small,other,full,about_the_same,no,no,yes,NaN,NaN,NaN,NaN,0.0,5.0,NaN,NaN,NaN
2384,trade,Small,NaN,NaN,NaN,yes,yes,yes,NaN,NaN,NaN,20.0,NaN,NaN,NaN,1.0,NaN


In [14]:
frequency_table = data_frame_of_small_workplaces_that_allow_employees_to_work_from_home.stb.freq(['premiums'])
frequency_table

,premiums,count,percent,cumulative_count,cumulative_percent
0,full,324,46.285714,324,46.285714
1,partial,310,44.285714,634,90.571429
2,no,66,9.428571,700,100.000000


In [15]:
summary_of_missing_values = data_frame_of_small_workplaces_that_allow_employees_to_work_from_home.stb.missing()
summary_of_missing_values.loc['premiums'].to_frame()

,premiums
missing,9.000000
total,709.000000
percent,1.269394


## Problem 13
Anything that can be done in SQL can be done with `pandas`. The next several questions ask you to write `pandas` code to match a given SQL query. But to check that the SQL query and `pandas` code yield the same result, create a new database using the `sqlite3` package and input the cleaned WHA data as a table in this database. (See module 6 for a discussion of SQlite in Python.) [1 point]

In [16]:
def query_database(database, query):
    cursor = WHA_survey_db.cursor()
    cursor.execute(query)
    data_frame_without_column_names = cursor.fetchall()
    column_names = [x[0] for x in cursor.description]
    data_frame = pd.DataFrame(data_frame_without_column_names, columns = column_names)
    return data_frame
    
import sqlite3
WHA_survey_db = sqlite3.connect('WHA_Survey.db')
data_frame.to_sql('curated_WHA_survey_data', WHA_survey_db, index = False, chunksize = 1000, if_exists = 'replace')
query_database(WHA_survey_db, 'SELECT * FROM curated_WHA_survey_data')

,industry,employee_size,organization_type,premiums,premium_change,part_time_insurance_coverage,health_promotion,work_from_home,percent_under30,percent_over60,percent_female,percent_hourly,percent_atypical_shift,percent_remote,percent_union,percent_turnover,gender_balance
0,health_and_social,Large,nonprofit,partial,about_the_same,no,yes,yes,46.0,10.0,86.0,75.0,50.0,50.0,0.0,35.0,Mostly women
1,health_and_social,Large,nonprofit,full,about_the_same,yes,yes,yes,45.0,25.0,65.0,60.0,60.0,5.0,NaN,7.0,Balanced
2,health_and_social,Large,state_or_local_government,full,about_the_same,no,yes,no,40.0,20.0,75.0,30.0,5.0,0.0,70.0,NaN,Mostly women
3,health_and_social,Large,nonprofit,partial,about_the_same,yes,no,no,40.0,40.0,60.0,40.0,NaN,NaN,50.0,NaN,Balanced
4,health_and_social,Large,nonprofit,full,smaller,yes,no,no,40.0,20.0,65.0,60.0,100.0,5.0,0.0,25.0,Balanced
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2837,trade,Small,for_profit_private,full,about_the_same,yes,no,no,NaN,2.0,2.0,90.0,0.0,0.0,0.0,0.0,Mostly men
2838,trade,Small,nonprofit,full,about_the_same,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2839,trade,Small,for_profit_private,partial,larger,yes,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2840,trade,Small,for_profit_public,full,larger,no,yes,yes,NaN,2.0,15.0,NaN,NaN,90.0,0.0,15.0,Mostly men


## Problem 14
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT size, type, premiums AS insurance, percent_female FROM whpps
WHERE industry = 'Hospitals' AND premium_change='Smaller'
ORDER BY percent_female DESC;
```
For each of these queries, your feature names might be different from the ones listed in the query, depending on the names you chose in problem 3.
[2 points]

In [17]:
query = '''
SELECT employee_size, organization_type, premiums AS insurance, percent_female
FROM curated_WHA_survey_data
WHERE industry = 'hospitals' AND premium_change='smaller'
ORDER BY percent_female DESC;
'''
result = query_database(WHA_survey_db, query)
result = result.replace([None], np.NaN)
result

,employee_size,organization_type,insurance,percent_female
0,Medium,nonprofit,full,89.0
1,Large,nonprofit,partial,80.0
2,Large,nonprofit,partial,80.0
3,Small,nonprofit,full,75.0
4,Medium,nonprofit,partial,65.0
5,Medium,for_profit_private,full,50.0
6,Large,nonprofit,partial,NaN
7,Medium,nonprofit,full,NaN
8,Medium,NaN,partial,NaN
9,Medium,nonprofit,partial,NaN


In [18]:
filter = (data_frame['industry'] == 'hospitals') & (data_frame['premium_change'] == 'smaller')
columns = ['employee_size', 'organization_type', 'premiums', 'percent_female']
slice = data_frame.loc[filter, columns]
slice = slice.rename(columns = {'premiums': 'insurance'})
slice = slice.sort_values(by = 'percent_female', ascending = False)
slice = slice.reset_index(drop = True)
slice

,employee_size,organization_type,insurance,percent_female
0,Medium,nonprofit,full,89.0
1,Large,nonprofit,partial,80.0
2,Large,nonprofit,partial,80.0
3,Small,nonprofit,full,75.0
4,Medium,nonprofit,partial,65.0
5,Medium,for_profit_private,full,50.0
6,Large,nonprofit,partial,NaN
7,Medium,nonprofit,full,NaN
8,Medium,NaN,partial,NaN
9,Medium,nonprofit,partial,NaN


## Problem 15
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT industry, 
    AVG(percent_female) as percent_female, 
    AVG(percent_under30) as percent_under30,
    AVG(percent_over60) as percent_over60
FROM whpps
GROUP BY industry
ORDER BY percent_female DESC;
```
[2 points]

In [19]:
query = '''
SELECT industry, AVG(percent_female) as percent_female, AVG(percent_under30) as percent_under30, AVG(percent_over60) as percent_over60
FROM curated_WHA_survey_data
GROUP BY industry
ORDER BY percent_female DESC;
'''
query_database(WHA_survey_db, query)

,industry,percent_female,percent_under30,percent_over60
0,health_and_social,80.657143,25.745665,11.349570
1,hospitals,76.427027,27.213793,16.489655
2,pleasure,53.804416,38.566343,11.544872
3,technology_economy_and_management,50.632184,23.821752,12.465465
4,public_administration,39.056738,21.015625,15.015385
5,trade,32.657258,29.108696,12.584034
6,resources_and_infrastructure,20.328605,22.257143,10.690355


In [20]:
summary = data_frame.groupby('industry').agg({'percent_female':'mean', 'percent_under30':'mean', 'percent_over60':'mean'})
summary = summary.sort_values(by = 'percent_female', ascending = False)
summary = summary.reset_index()
summary

,industry,percent_female,percent_under30,percent_over60
0,health_and_social,80.657143,25.745665,11.349570
1,hospitals,76.427027,27.213793,16.489655
2,pleasure,53.804416,38.566343,11.544872
3,technology_economy_and_management,50.632184,23.821752,12.465465
4,public_administration,39.056738,21.015625,15.015385
5,trade,32.657258,29.108696,12.584034
6,resources_and_infrastructure,20.328605,22.257143,10.690355


## Problem 16
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT gender_balance, premiums, COUNT(*)
FROM whpps
GROUP BY gender_balance, premiums
HAVING gender_balance is NOT NULL and premiums is NOT NULL;
```
[2 points]

In [21]:
query = '''
SELECT gender_balance, premiums, COUNT(*)
FROM curated_WHA_survey_data
GROUP BY gender_balance, premiums
HAVING gender_balance is NOT NULL and premiums is NOT NULL
'''
query_database(WHA_survey_db, query)

,gender_balance,premiums,COUNT(*)
0,Balanced,full,226
1,Balanced,no,77
2,Balanced,partial,271
3,Mostly men,full,301
4,Mostly men,no,91
5,Mostly men,partial,332
6,Mostly women,full,267
7,Mostly women,no,107
8,Mostly women,partial,333


In [22]:
summary = data_frame.groupby(['gender_balance', 'premiums']).size()
summary = summary.reset_index()
summary = summary.rename(columns = {0: 'COUNT(*)'})
summary['gender_balance'] = pd.Categorical(summary['gender_balance'], ['Balanced', 'Mostly men', 'Mostly women'])
summary = summary.sort_values(by = 'gender_balance')
summary = summary.reset_index(drop = True)
summary

,gender_balance,premiums,COUNT(*)
0,Balanced,full,226
1,Balanced,no,77
2,Balanced,partial,271
3,Mostly men,full,301
4,Mostly men,no,91
5,Mostly men,partial,332
6,Mostly women,full,267
7,Mostly women,no,107
8,Mostly women,partial,333
